In [ ]:
# # mypy: ignore-errors
# from neural_bandits.modules.linear_bandit_module import LinearBanditModule
# from neural_bandits.algorithms.linear_bandits import LinearUCBBandit
# from neural_bandits.benchmark.datasets.statlog import StatlogDataset

# import lightning as pl
# from torch.utils.data import DataLoader
# import torch

In [ ]:
# # mypy: ignore-errors
# dataset = StatlogDataset()
# print(dataset.context_size)
# print(len(dataset))

# train_loader = DataLoader(dataset, batch_size=1, shuffle=True)
# model = LinearBanditModule(
#         linear_bandit_type = LinearUCBBandit,
#         n_features = dataset.context_size * dataset.num_actions,
# )
# logger = pl.pytorch.loggers.CSVLogger("logs", name="linear_bandit", flush_logs_every_n_steps=100)
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

/hpi/fs00/home/robert.weeke/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /hpi/fs00/home/robert.weeke/miniconda3/envs/neural_b ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


7
58000


In [ ]:
# # mypy: ignore-errors

# # TODO: Create a bunch of selectors in the utils
# Selector = lambda x: x.argmax(dim=1)

In [ ]:
# TODO: Adjust the training_step method of each module to take:
#  - tuple of (chosen_contextualized_actions [batch_size, num_chosen_actions, num_features], realized_reward [batch_size, ])

In [ ]:
# # mypy: ignore-errors

# # USER CODE ("Online" Learning with Delayed Feedback from Logged Data of Linear Models)
# # dataset = Dataset.LoadFromDB()  # Bandit Feedback: this contains the last +- 1000 samples but we don't know the exact size
# train_loader = DataLoader(dataset, batch_size=32, shuffle=True) # this loads a bunch of mini-batches. we dont know how big the dataset is.
# trainer.fit(model, train_loader)

In [ ]:
# # mypy: ignore-errors

# # TODO: make nice classes
# class LinearBenchmark:
#     def __init__(self, model, trainer, train_loader):
#         self.model = model
#         self.trainer = trainer
#         self.train_loader = train_loader # MNIST of 100000 samples, split into mini-batches of 32 (or 1)

#     def run(self):
#         for batch_idx, (contextualized_actions, rewards) in enumerate(self.train_loader):
#             action_distribution = self.model(contextualized_actions)
#             actions = Selector(action_distribution)
#             realized_rewards = rewards[torch.arange(len(rewards)), actions]
#             chosen_actions = contextualized_actions[torch.arange(len(contextualized_actions)), actions]
#             self.on_batch(chosen_actions, realized_rewards)

#     def on_batch(
#         self,
#         chosen_actions: torch.Tensor,  # shape (batch_size, num_chosen_actions)
#         realized_rewards: torch.Tensor,  # shape (batch_size,)
#     ):
#         batch_dataset = torch.utils.data.TensorDataset(chosen_actions, realized_rewards)
#         batch_data_loader = torch.utils.data.DataLoader(
#             batch_dataset,
#             batch_size=len(batch_dataset),
#         )
#         self.trainer.fit(self.model, batch_data_loader)


In [ ]:
# # mypy: ignore-errors

# # USER CODE ("Online" Learning with Delayed Feedback from Logged Data)
# # dataset = Dataset.LoadFromDB()  # Bandit Feedback: this contains the last +- 1000 samples but we don't know the exact size
# train_loader = DataLoader(dataset, batch_size=32, shuffle=True) # this loads a bunch of mini-batches. we dont know how big the dataset is.
# trainer.fit(model, train_loader)

# Neural UCB

In [ ]:
# # mypy: ignore-errors
# class NeuralUCBBenchmark:
#     def __init__(self, model, trainer, train_loader):
#         self.model = model
#         self.trainer = trainer
#         self.train_loader = train_loader
#         self.mini_batch_size = 32

#     def run(self):
#         for batch_idx, (contextualized_actions, rewards) in enumerate(self.train_loader):
#             action_distribution = self.model(contextualized_actions)
#             actions = Selector(action_distribution)
#             realized_rewards = rewards[torch.arange(len(rewards)), actions]
#             chosen_actions = contextualized_actions[torch.arange(len(contextualized_actions)), actions]
#             self.on_batch(chosen_actions, realized_rewards)
            
#             if batch_idx % 100 == 0:
#                 self.on_interval()

#     def on_batch(
#         self,
#         chosen_actions: torch.Tensor,  # shape (batch_size, num_chosen_actions)
#         realized_rewards: torch.Tensor,  # shape (batch_size,)
#     ):
#         # option 2
#         self.add_data(chosen_actions, realized_rewards)

#     def on_interval(
#         self,
#     ):
#         # option 2
#         data_loader = DataLoader(Dataset(self.data))
#         self.trainer.fit(self.model, data_loader)


In [ ]:

# # USER CODE: NeuralUCB from delayed feedback. OPTION 1: Bandit decides itself when to update
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

# model = NeuralUCB(
#     initial_training_steps=1000,  # number of data points
#     update_freq=1000, # number of data points
#     train_data_strategy=AllDataStrategy() | SlidingWindowDataStrategy(data_points=100), # last = this data
#     warm_start=True,nn=torch.nn.Sequential([])
# )

# # dataloader: each batch: 32 * (chosen_action, realized_reward)
# # mini-batch 1: {(x, y), (x, y), ... 32 times} -> maybe update Neural Network, store batch in storage
# # mini-batch 2: {(x, y), (x, y), ... 32 times} -> maybe update Neural Network (can use last batch as well by loading from storage)
# # ...

# # for existing dataset
# trainer.fit(model, dataloader) # sometimes it doesn't do anything, once there is enough new data it will update the model

# # training + inference at once loop
# logger = Logger()
# for i in range(1000):
#     actions = model(data)
#     rewards = environment.get_rewards(actions)
#     trainer = Trainer(logger) # unfortunately, we need to create a new trainer every time because it can't handle being called with a different data loader
#     trainer.fit(model, DataLoader(Dataset((actions, rewards))))
#     # -> can't use batch_idx
#     # -> need a warning if the model hasn't updated
    
# # INFERENCE LOOP
# actions = model(context)
# # return to user

# # STREAM OF USER REWARDS
# database.add_reward(action, realized_reward)

# # TRAINER
# trainer.fit(model, DataLoader(Dataset(database.get_rewards())))

## Other possible interfaces we do not use

In [ ]:
# ########################################### OLD ##########################################################
# # USER CODE: NeuralUCB from delayed feedback. OPTION 2: Bandit stores data and user decides when to update
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

# model = NeuralUCB(
#     update_freq=None,
#     train_data_strategy=DataStrategy.ALL | DataStrategy.SLIDING_WINDOW | DataStrategy.LAST_DATA,
# )

# # dataset: n * (chosen_action, realized_reward)
# # throughout the day
# model.add_data(dataset)

# # on midnight or when enough data is collected
# trainer.fit(model) # loads the data from self.train_dataloader() automatically (depending on the data strategy)

In [ ]:
# # USER CODE: NeuralUCB from delayed feedback. OPTION 3: User stores data and user decides when to update
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

# model = NeuralUCB(
#     update_freq=None,
#     train_data_strategy=DataStrategy.LAST_DATA,
# )

# # data_loader: n batches of size m of (chosen_action, realized_reward)
# trainer.fit(model, your_dataloader) # loads the data from self.train_dataloader() automatically (depending on the data strategy)

In [ ]:
# another function for retrieving from storage
# your_data = model.get_data() or model.train_dataloader()

In [ ]:
# # USER CODE: NeuralLinear from delayed feedback
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

# model = NeuralLinear(
#     update_encoder_freq=1000,
#     update_head_freq=64,
#     train_data_strategy=DataStrategy.ALL | DataStrategy.SLIDING_WINDOW | DataStrategy.MINI_BATCH,
# )

# # train_loader: each batch: 32 * (chosen_action, realized_reward)
# # mini-batch 1: {(x, y), (x, y), ... 32 times} -> update head, store batch in storage
# # mini-batch 2: {(x, y), (x, y), ... 32 times} -> update head (can use last batch as well by loading from storage?)
# # ...

# # model.set_update_freq(100)
# model.retrain()
# trainer.fit(model, train_loader)

# storage_loader = DataLoader(storage)
# model.force_retrain() # from storage
# trainer.fit(model, storage_loader)


# # for batch_idx, (contextualized_actions, rewards) in enumerate(train_loader):

# #     if batch_idx % 100 != 0:
# #         batch_dataset = torch.utils.data.TensorDataset(contextualized_actions, rewards)
# #         trainer.fit(model, batch_dataset)
# #     else:
        

# Neural Linear

In [ ]:
# class NeuralLinearBenchmark:
#     def __init__(self, model, trainer, train_loader):
#         self.model = model
#         self.trainer = trainer
#         self.train_loader = train_loader
#         self.mini_batch_size = 32

#     def run(self):
#         for batch_idx, (contextualized_actions, rewards) in enumerate(self.train_loader):
#             action_distribution = self.model(contextualized_actions)
#             actions = Selector(action_distribution)
#             realized_rewards = rewards[torch.arange(len(rewards)), actions]
#             chosen_actions = contextualized_actions[torch.arange(len(contextualized_actions)), actions]
#             self.on_batch(chosen_actions, realized_rewards)
            
#             if batch_idx % 100 == 0:
#                 self.on_interval()

#     def on_batch(
#         self,
#         chosen_actions: torch.Tensor,  # shape (batch_size, num_chosen_actions)
#         realized_rewards: torch.Tensor,  # shape (batch_size,)
#     ):
#         batch_dataset = torch.utils.data.TensorDataset(chosen_actions, realized_rewards)
#         batch_data_loader = torch.utils.data.DataLoader(
#             batch_dataset,
#             batch_size=len(batch_dataset),
#         )
#         self.trainer.fit(self.model, batch_data_loader)

#     def on_interval(
#         self,
#     ):
#         self.model.train_encoder()
#         self.trainer.fit(self.model)


In [ ]:
# ### PROPOSAL (rob2u):
# # IDK if this will work, although I do not see a reason why it shouldnt.
# # The issue is that we require the user to perform the update of the head and the encoder manually.

# class BanditTrain(pl.LightningModule):
#     def __init__(self, model):
#         self.neural_net = model.neural_net
#         self.head = model.head
#         self.mini_batch_size = 32
        
#     def forward(self, x):
#         return self.model(x)
    
#     def on_train_epoch_start(self):
#         self.head.reset()
    
#     def training_step(self, batch, batch_idx):
#         chosen_action_contexts, realized_rewards = batch
        
#         # send through feature extractor (neural network)
#         x = self.feature_extractor(chosen_action_contexts)
        
#         # send through head
#         self.head.update(x, realized_rewards)
        
#         return torch.tensor(0.0)


# class NeuralLinearBenchmark:
#     def __init__(self, model, trainer_linear_bandit, trainer_neural_net, train_loader):
#         self.model = model
#         self.trainer_neural_net = trainer_neural_net
        
#         self.bandit_train_module = BanditTrain(model)
#         self.trainer_linear_bandit = trainer_linear_bandit

#         self.train_loader = train_loader
#         self.mini_batch_size = 32

#     def run(self):
#         for batch_idx, (contextualized_actions, rewards) in enumerate(self.train_loader):
#             action_distribution = self.model(contextualized_actions)
#             actions = Selector(action_distribution)
#             realized_rewards = rewards[torch.arange(len(rewards)), actions]
#             chosen_actions = contextualized_actions[torch.arange(len(contextualized_actions)), actions]
#             self.on_batch(chosen_actions, realized_rewards)
            
#             if batch_idx % 100 == 0:
#                 self.on_interval()

#     def on_batch(
#         self,
#         chosen_actions: torch.Tensor,  # shape (batch_size, num_chosen_actions)
#         realized_rewards: torch.Tensor,  # shape (batch_size,)
#     ):
#         batch_dataset = torch.utils.data.TensorDataset(chosen_actions, realized_rewards)
#         batch_data_loader = torch.utils.data.DataLoader(
#             batch_dataset,
#             batch_size=len(batch_dataset),
#         )
#         self.trainer_linear_bandit.fit(self.bandit_train_module, self.model.train_dataloader())

#     def on_interval(
#         self,
#     ):
#         self.trainer.fit(self.model)
#         self.trainer_linear_bandit.fit(self.bandit_train_module, self.model.train_dataloader())


In [ ]:
# ### PROPOSAL (philipp)
# import torch.nn as nn

# class BanditNetwork(pl.LightningModule):
#     def __init__(self, model):
#         self.model = model

#     def forward(self, x):
#         return self.model(x)

#     def training_step(self, batch, batch_idx):
#         chosen_action_contexts, realized_rewards = batch
        
#         z = self.forward(chosen_action_contexts)
#         loss = nn.functional.mse_loss(z, realized_rewards)
        
#         return loss

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=1e-3)

# class NeuralLinearBenchmark:
#     def __init__(self, encoder_model, trainer_linear_bandit, trainer_neural_net, train_loader):
#         self.model = NeuralLinear(BanditNetwork(encoder_model))
#         self.trainer_neural_net = trainer_neural_net
#         self.trainer_linear_bandit = trainer_linear_bandit

#         self.train_loader = train_loader
#         self.mini_batch_size = 32

#     def run(self):
#         for batch_idx, (contextualized_actions, rewards) in enumerate(self.train_loader):
#             action_distribution = self.model(contextualized_actions)
#             actions = Selector(action_distribution)
#             realized_rewards = rewards[torch.arange(len(rewards)), actions]
#             chosen_actions = contextualized_actions[torch.arange(len(contextualized_actions)), actions]
#             self.on_batch(chosen_actions, realized_rewards)
            
#             if batch_idx % 100 == 0:
#                 self.on_interval()

#     def on_batch(
#         self,
#         chosen_actions: torch.Tensor,  # shape (batch_size, num_chosen_actions)
#         realized_rewards: torch.Tensor,  # shape (batch_size,)
#     ):
#         batch_dataset = torch.utils.data.TensorDataset(chosen_actions, realized_rewards)
#         batch_data_loader = torch.utils.data.DataLoader(
#             batch_dataset,
#             batch_size=len(batch_dataset),
#         )
#         # using option 2a
#         self.trainer_linear_bandit.fit(self.model, batch_data_loader)

#     def on_interval(
#         self,
#     ):
#         self.trainer_neural_net.fit(self.model.encoder, self.model.train_dataloader())
#         self.model.update_encoder_data() # recompute all encodings. TODO: Can we compute them "lazy"? e.g. by invalidating them: self.model.invalidate_encodings()
#         self.trainer_linear_bandit.fit(self.model, self.model.encoder.train_dataloader()) # IMPORTANT to use the data from the encoder because only it contains the pairs of (chosen_action, realized_reward)?


In [ ]:
# ### USER CODE (philipp): OPTION 1: Bandit decides itself when to update
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

# model = NeuralLinear(
#     encoder = BanditNetwork(encoder_model),
#     update_encoder_freq=1000,
#     update_head_freq=64,
#     train_data_strategy=DataStrategy.ALL | DataStrategy.SLIDING_WINDOW | DataStrategy.MINI_BATCH,
# )

# # train_loader: each batch: 32 * (chosen_action, realized_reward)
# # mini-batch 1: {(x, y), (x, y), ... 32 times} -> update head, store batch in storage
# # mini-batch 2: {(x, y), (x, y), ... 32 times} -> update head (can use last batch as well by loading from storage?)
# # ...

# # Optional overwrite of the update frequencies
# # model.set_update_encoder_freq(100)
# # model.set_update_head_freq(32)

# trainer.fit(model, train_loader)
# # 1. adds the data to the storage
# # 2. if train_encoder_freq is reached, trains the encoder on pairs of context_actions and rewards. Then invalidates the encodings/retrains the head on pairse of updated embeddings and rewards
# # 3. elif train_head_freq is reached, trains the head on pairs of embeddings and rewards

In [ ]:
# # USER CODE: NeuralLinear from delayed feedback. OPTION 2: Bandit stores data and user decides when to update
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)
# encoder_trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

# model = NeuralLinear(
#     encoder = BanditNetwork(encoder_model),
#     update_encoder_freq=None,
#     update_head_freq=None,
#     train_data_strategy=DataStrategy.ALL | DataStrategy.SLIDING_WINDOW | DataStrategy.LAST_DATA,
# )

# # dataset: n * (chosen_action, realized_reward)
# # throughout the day
# model.add_data(dataset)

# # on midnight or when enough data is collected
# trainer.fit(model) # loads the data from self.train_dataloader() automatically (depending on the data strategy)
# # 1. data already in storage, so no need to add it again (but we need some sort of ID!)
# # 2. trains the head on pairs of embeddings and rewards. if any data is invalidated -> also call the encoder and save result

# # rarely update the encoder
# encoder_trainer.fit(model.encoder, model.encoder.train_dataloader())
# # 1. data already in storage, so no need to add it again
# # 2. trains the encoder on pairs of context_actions and rewards.
# # 3. Then invalidate the embeddings. TODO QUESTION? Can we do this automatically after training the encoder? Would need a reference to the NeuralLinear model...
# # Otherwise: user needs to call: model.invalidate_encodings()

In [ ]:
# # USER CODE: NeuralLinear from delayed feedback. OPTION 2a: Bandit stores data and user decides when to update but only for network
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)
# encoder_trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

# model = NeuralLinear(
#     encoder = BanditNetwork(encoder_model),
#     update_encoder_freq=None,
#     update_head_freq=16,
#     train_data_strategy=DataStrategy.ALL | DataStrategy.SLIDING_WINDOW | DataStrategy.LAST_DATA,
# )

# # train_loader: each batch: 32 * (chosen_action, realized_reward)
# # mini-batch 1: {(x, y), (x, y), ... 32 times} -> update head, store batch in storage
# # mini-batch 2: {(x, y), (x, y), ... 32 times} -> update head (can use last batch as well by loading from storage?)
# # ...
# trainer.fit(model, train_loader)
# # 1. adds the data to the storage
# # 2. if train_head_freq is reached, trains the head on pairs of embeddings and rewards. if any data is invalidated -> also call the encoder and save result

# # on midnight or when enough data is collected
# encoder_trainer.fit(model.encoder)
# # 1. data already in storage, so no need to add it again
# # 2. trains the encoder on pairs of context_actions and rewards.
# # 3. Then invalidate the embeddings. TODO QUESTION? Can we do this automatically after training the encoder? Would need a reference to the NeuralLinear model...
# # Otherwise: user needs to call: model.invalidate_encodings()

In [ ]:
# USER CODE: NeuralLinear from delayed feedback. OPTION 2b: Bandit stores data and user decides when to update but only for head. Seems to be rare...

# NOT SUPPORTED!
# That would require us to update the encoder in "add_data".

In [ ]:
# # USER CODE: NeuralUCB from delayed feedback. OPTION 3: User stores data and user decides when to update
# trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)
# encoder_trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

# model = NeuralUCB(
#     encoder = BanditNetwork(encoder_model),
#     update_encoder_freq=None,
#     update_head_freq=None,
#     train_data_strategy=DataStrategy.LAST_DATA,
# )

# # Either update head:
# # data_loader: n batches of size m of (chosen_action, realized_reward)
# trainer.fit(model, your_dataloader)
# # 1. TODO QUESTION: not sure if we should add the data anyways - even if we are in the setting where teh user stores the data. Can we differentiate between other cases? Don't think so...
# # 2. trains the head on pairs of embeddings and rewards


# # Or update encoder. Afterwards we need to update the head as well:
# encoder_trainer.fit(model, your_dataloader)
# # 1. trains the encoder on pairs of context_actions and rewards.
# # no need to call invalidate_encodings() because the bandit does not store the data itself
# your_data_loader.update_encodings(encoder_trainer) # smth like this is necessary!
# trainer.fit(model, your_dataloader)
# # 1. trains the head on pairs of embeddings and rewards

In [ ]:
# IF WE DETECT THAT NOT ENOUGH DATA WAS IN THE DATASET, WE THROW A WARNING!